In [0]:


date = "2020-03-22"

for device_code in ["android-all","ios-all","ios-phone","ios-tablet"]:
    unified_s3_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/granularity=daily/date={}/device_code={}/"
    print spark.read.format("delta").load(unified_s3_path.format(date, device_code)).groupBy('country_code').count().orderBy('country_code', ascending=True)


In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/granularity=daily/date=2020-05-22/
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/date=2020-08-08/platform=android/

In [0]:
%%sh

PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 

--SELECT date,granularity,count(*) FROM store.store_est_publisher_fact_v2  where date between '2020-06-30' and '2020-06-30' and est_free_app_download is not null group by date,granularity;
--SELECT date,granularity,count(*) FROM store.store_est_publisher_fact_v2  where date between '2020-06-30' and '2020-06-30' and est_paid_app_download is not null group by date,granularity;
--SELECT date,granularity,count(*) FROM store.store_est_publisher_fact_v2  where date between '2020-06-30' and '2020-06-30' and est_revenue is not null group by date,granularity;

-- SELECTgranularity) FROM store.store_est_publisher_fact_v2  where date between '2020-06-30' and '2020-06-30' and est_revenue is not null;

-- SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_fact_v2  where date between '2020-01-01' and '2020-01-31';
-- SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_fact_v2  where date between '2020-02-01' and '2020-02-29';
-- SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_fact_v2  where date between '2020-03-01' and '2020-03-31';
-- SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_fact_v2  where date between '2020-04-01' and '2020-04-30';
-- SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_fact_v2  where date between '2020-05-01' and '2020-05-31';

--SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_t_m_fact_v2  where date between '2020-01-01' and '2020-01-31';

--SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_t_w_fact_v2  where date between '2020-01-01' and '2020-01-31';

--SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_t_q_fact_v2  where date between '2020-01-01' and '2020-01-31';

--SELECT sum(est_revenue),count(est_revenue) FROM store.store_est_publisher_t_y_fact_v2  where date between '2020-01-01' and '2020-01-31';

--SELECT sum(est_revenue) FROM store.store_est_publisher_t_w_fact_v2  where date between '2010-09-01' and '2010-09-04';
--SELECT sum(est_revenue) FROM store.store_est_publisher_t_w_fact_v2  where date between '2010-09-05' and '2010-09-11';
--SELECT sum(est_revenue) FROM store.store_est_publisher_t_w_fact_v2  where date between '2010-09-12' and '2010-09-18';


SELECT sum(est_revenue) FROM store.store_est_publisher_t_m_fact_v2 WHERE granularity='monthly' AND date BETWEEN '2020-07-01' AND '2020-07-31';
SELECT sum(est_revenue) FROM store.store_est_publisher_fact_v2 WHERE granularity='daily' AND date BETWEEN '2020-07-01' AND '2020-07-31';

SELECT sum(est_revenue) FROM store.store_est_publisher_t_w_fact_v2 WHERE granularity='weekly' AND date BETWEEN '2020-08-16' AND '2020-08-22';
SELECT sum(est_revenue) FROM store.store_est_publisher_fact_v2 WHERE granularity='daily' AND date BETWEEN '2020-08-16' AND '2020-08-22';

EOF


In [0]:
%%sh

PGPASSWORD='IEdwiF2hmqzdKK43' psql -h b2b-prod-uds-storage-meta-db-new.crlexxwtzodp.us-east-1.rds.amazonaws.com  -U app_meta_qa -d metadb -p 5432 << EOF 
\x
select app_stats -> 'latest_available_date' as latest_available_date from meta where table_name like '%store_est_publisher%' and is_active = True ;
--select * from meta where table_name like '%store_est_publisher%' and is_active = True ;
--select data_mart,table_name,version,vendor from meta where table_name like '%store_est_publisher%';


--SELECT * FROM meta;
    

EOF




In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate
    from ms.market_size_daily_estimate_1000 
    where 
        date = '2020-01-10' and 
        purchase_type_id=12 and 
        app_type_id=2 and 
        store_id=14 and 
        category_id in (65) 
\$proxy\$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);
EOF



In [0]:
%%sh

aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

select date, sum(count_a) from plproxy.execute_select_nestloop(\$proxy\$ 
    select date, count(*) as count_a
    from mu.app_daily
    where 
        date between '2020-07-18' and  '2020-08-18'
    group by date
\$proxy\$) tpl (date DATE, count_a BIGINT) group by date order by date asc;

EOF



In [0]:
%%sh
echo  2020-08-09
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/date=2020-08-09/platform=android/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/granularity=daily/date=2020-08-09/device_code=android-all/
echo 2020-08-08
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/date=2020-08-08/platform=android/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/granularity=daily/date=2020-08-08/device_code=android-all/


In [0]:
%%sh
